# Unicycle Control Examples

This code illustrates how we integrate the theory from class into computational models.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.integrate import solve_ivp
from math import cos, sin, pi, atan2, dist
from functools import partial
import sympy as sym

We will redefine our unicycle dynamics function.

In [ ]:
# I provide by default no control input.
def unicycle_motion(t, x, u_fn=lambda t, x : [0,0]):
    '''
    Parameters:
    t : float
        current time
    x : list(float)
        current state
    u_fn : function
        control function; defaults to no control input
    '''
    [v, omega] = u_fn(t, x)
    theta = x[2]
    x_dot = v*cos(theta)
    y_dot = v*sin(theta)
    theta_dot = omega
    return [x_dot, y_dot, omega]

In [ ]:
# Let's plot the (x,y) evolution of the unicycle
def plot_xy_trajectory(X, xlim=None, ylim=None):
    fig,ax = plt.subplots(1)
    if xlim is not None:
        ax.set_xlim(xlim)
    if ylim is not None:
        ax.set_ylim(ylim)
    plt.plot(X[0,:], X[1,:], 'b--')
    plt.plot(0., 0., 'k*')
    plt.plot(X[0,-1], X[1,-1], 'go')

## Proportional Control GoToGoal

This is the simplest control algorithm that just uses the error signal of $e(t)=\theta_g(t) - \theta_R(t)$ to change how it moves.
Once the vehicle gets close to the goal, it stops.

In [ ]:
def simple_gotogoal(t, x, x_g=None, v_0=0.5, k=1.0, eps=0.1):
    '''
    This function takes a proportional constant, k, to affect the strength
    of the correction. You also supply some constant forward, v0.
    '''
    if x_g is not None:
        # Set the forward speed
        if dist(x[0:2], x_g[0:2]) > eps:
            u_1 = v_0
        else:
            u_1 = 0.0
        # theta = x[2]!
        u_2 = k*(atan2(x_g[1]-x[1], x_g[0]-x[0]) - x[2])
        return [u_1, u_2]
    else:
        # No goal provided. Do nothing.
        return [0., 0.]

In [ ]:
# Pick the goal location and create the partial function
u_gtg = partial(simple_gotogoal, x_g=[2.0, 1.0, 0.0])

In [ ]:
# Simulate
model_fn = partial(unicycle_motion, u_fn=u_gtg)
# Final time
t_f = 5.0 
t_span = (0.0, t_f)
# Initial state of the vehicle - the origin
x_0 = [0., 0., pi/2+0.4]
soln = solve_ivp( model_fn, t_span, x_0, method='RK23', dense_output=False )

In [ ]:
plot_xy_trajectory(soln.y)

In [ ]:
plt.plot(soln.t, soln.y[0], soln.t, soln.y[1])
plt.xlabel('Time (s)')

## Nonlinear GoToGoal

This method was developed using Lyapunov theory and is proven to drive the robot any $x_g\in X$.
It will sometimes be slower than the naive proportional controller.

In [ ]:
# A more elegant nonlinear controller
def nonlinear_gotogoal(t, x, x_g=None, k=1.0, eps=0.1):
    if x_g is not None:
        d = dist(x[0:2], x_g[0:2])
        theta_g = atan2(x_g[1]-x[1], x_g[0]-x[0])
        if d < 0.1:
            return [0., 0.]
        else:
            v = k*d*cos(theta_g - x[2])
            omega = k*d*sin(theta_g - x[2])
        return [v, omega]
    else:
        return [0.,0.]    

In [ ]:
u_gtg2 = partial(nonlinear_gotogoal, x_g=[2.0, 1.0, 0.0], k=2.0)
model_fn2 = partial(unicycle_motion, u_fn=u_gtg2)

In [ ]:
x_0 = [0., 0., pi/2+0.4]
t_span=(0,5)
soln = solve_ivp( model_fn2, t_span, x_0, method='RK23', dense_output=True )

In [ ]:
plot_xy_trajectory(soln.y)

In [ ]:
plt.plot(soln.t, soln.y[0], soln.t, soln.y[1])
plt.xlabel('Time (s)')

## Linearized Unicycle

It is nice to have these other controllers, but would be better is to abstract the whole unicycle as a single point!
To do so, we derived a result in class tha enables us to create linear controllers that move the point, rather than deal with the complexities of a nonlinear model.
The tradeoff is some tracking error.

In [ ]:
def linearized_unicycle_motion(t, x, l=0.02, u_fn=lambda t, x : [0,0]):
    '''
    This function consumes a 2D control vector that assumes the control of a point.
    You provide the offset length, l. In the default, it would be 5cm  from the center.
    '''
    x1 = x[0]
    x2 = x[1]
    x3 = x[2] # theta
    u = np.array(u_fn(t, x)) # Original control inputs
    # Mapping to v, omega
    u_map = np.linalg.inv([[cos(x3), -l*sin(x3)],[sin(x3), l*cos(x3)]]).dot(u)
    x_dot = u_map[0]*cos(x3)
    y_dot = u_map[0]*sin(x3)
    theta_dot = u_map[1]
    return [x_dot, y_dot, theta_dot]

In [ ]:
x_0 = [0., 0., 0.]
t_span=(0,5)
model_fn3 = partial(linearized_unicycle_motion, u_fn=lambda t,x : [0.1, 0.1])
soln = solve_ivp( model_fn3, t_span, x_0, method='RK23', dense_output=True )

In [ ]:
plot_xy_trajectory(soln.y)

In [ ]:
plt.plot(soln.t, soln.y[0], soln.t, soln.y[1])
plt.xlabel('Time (s)')

## Unicycle Trajectory Following

This code shows you how to create a state feedback controller that tracks a provided reference trajectory.
At this point, all we need is a description of the curve we want to track, $(x_{ref}(t), y_{ref}(t))$, and we place the poles (e.g. state feedback) anywhere on the left half plane.
**Why?** A key concept is reducing our error to 0 over time: $e(t)=x_{ref}(t)-x(t)\rightarrow 0$.
Take the derivative to give us:
$$
\dot{e} = \dot{x}_{ref}-\dot{x} = \begin{bmatrix} \dot{x}_{ref} - u_1 \\ \dot{y}_{ref} - u_2 \end{bmatrix}
$$
We want to ensure that the error dynamics decay to 0. 
Thus the control inputs should structure the state feedback so the eigenvalues of the error dynamics are negative.

In [ ]:
def figure_eight(t):
    return [sin(t/5), sin(t/10)]

In [ ]:
# Take the derivative of the reference signal
t = sym.symbols('t')
dfig8x = sym.utilities.lambdify(t, sym.diff(sym.sin(t/5.0)))
dfig8y = sym.utilities.lambdify(t, sym.diff(sym.sin(t/10.0)))

In [ ]:
def dfigure_eight(t):
    return [dfig8x(t), dfig8y(t)]

We have everything we need to move forward! Now here is the trajectory tracking controller.

In [ ]:
def traj_tracking_controller(t, x, x_ref_fn, dx_ref_fn, k=[1, 1]):
    dx_ref = dx_ref_fn(t)
    x_ref = x_ref_fn(t)
    u_1 = dx_ref[0] + k[0]*(x_ref[0] - x[0])
    u_2 = dx_ref[1] + k[1]*(x_ref[1] - x[1])
    return [u_1, u_2]

In [ ]:
traj_track_fn = partial(traj_tracking_controller, x_ref_fn=figure_eight, dx_ref_fn=dfigure_eight, k=[0.5, 0.5])

In [ ]:
x_0 = [1., 0., pi/4]
t_f  = 100
t_span=(0,t_f)
model_fn4 = partial(linearized_unicycle_motion, u_fn=traj_track_fn)
soln = solve_ivp( model_fn4, t_span, x_0, method='RK23', dense_output=True )

In [ ]:
soln.t
num_pts = len(soln.t)

In [ ]:
ts = np.linspace(0,t_f,num_pts)
ref_traj = np.array([figure_eight(t) for t in ts])
ref_traj.shape

In [ ]:
plt.plot(ref_traj[:,0], ref_traj[:,1], 'r-.')
plt.plot(soln.y[0,:], soln.y[1,:], 'k--')